# Frontend Application interface to the Stabilization Models

At some point this could be converted to AWS CloudFormation but at this point understanding the infrastructure at a basic AWS SDK point is highly disirable.


## Frontend Setup


Setup the buckets associated with the application.  All the files exist either as code stored in the [project repository](https://github.com/mgrandau/hand-tracking-stabilization) or data files in the [application bucket](s3://mgrandau-springboard-capstone/frontend-application/)

The application is composed of 3 S3 buckets. One bucket contains the application and the canned input videos used for testing. A second bucket is used as an input bucke to the flow. The 3rd and last bucket contains the output of the stabilization python.


## Frontend Teardown

Cleanup the application buckets